In [1]:
import pandas as pd
import numpy as np
import os
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier


In [2]:
data_path = "/Users/allenouyang/jupyter/2023Spring ML&PA(MSU)/Assignment04/machine_learning_04_data" # replace with your own local folder path
train = pd.read_csv(os.path.join(data_path,"train_sample.csv"))
test = pd.read_csv(os.path.join(data_path,"test_sample.csv"))


To perform 5-fold cross-validation and find the best number of rounds for binary classification using XGBoost

In [3]:
dtrain = xgb.DMatrix(train.iloc[:,:-1], label=train['class'])
params = {'objective': 'binary:logistic', 'eval_metric': 'auc'}
cv_results = xgb.cv(params=params, dtrain=dtrain, num_boost_round=1000, nfold=5,
                    early_stopping_rounds=10, seed=0, verbose_eval=False)
best_num_rounds = cv_results.shape[0]


This code creates a DMatrix object from the training data, sets the parameters for the XGBoost model, and performs 5-fold cross-validation with a maximum of 1000 boosting rounds. The early_stopping_rounds parameter is used to stop boosting early if there is no improvement in performance for 10 rounds. The best number of rounds is determined from the cross-validation results.

To fit XGBoost with the best number of rounds and make predictions for the test set

In [6]:
bst = xgb.train(params=params, dtrain=dtrain, num_boost_round=best_num_rounds)
dtest = xgb.DMatrix(test)
prediction = bst.predict(dtest)


This code trains the XGBoost model with the best number of rounds on the entire training set and makes predictions for the test set.

To perform random forest classification with default settings and find the most important feature

In [7]:
rf = RandomForestClassifier(random_state=0)
rf.fit(train.iloc[:,:-1], train['class'])
importances = rf.feature_importances_
rf_most_important = np.argsort(importances)[-1] + 1


This code initializes a random forest classifier with random_state=0, fits it to the training data, and computes the feature importances. The index of the most important feature is determined and saved to rf_most_important.

In [8]:
np.savez('submission.npz', rf_most_important=rf_most_important, prediction=prediction)


In [11]:
npzfile = np.load('submission.npz')
print(npzfile['rf_most_important'],npzfile['prediction'].shape)

2 (500,)
